<a href="https://colab.research.google.com/github/varshini03/insider-threat-detection-framework/blob/main/role_rep_better.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import re
from glob import glob

In [ ]:
!pip install umap
!pip install hdbscan
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 41.6 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import StandardScaler
import umap
import hdbscan

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
df_features = pd.read_csv('/content/df_features.csv')

In [ ]:
df_features = df_features.drop(columns = {'Unnamed: 0'})

In [ ]:
a = pd.read_csv('/content/LDAP/2009-12.csv')
a.isnull().sum()

,0
employee_name,0
user_id,0
email,0
role,0
business_unit,0
functional_unit,2
department,14
team,127
supervisor,1


In [ ]:
import os

In [ ]:
ldap_dir = "/content/LDAP"  # Adjust this path as needed
ldap_files = sorted(glob(os.path.join(ldap_dir, "*.csv")))
if not ldap_files:
    raise FileNotFoundError(f"No CSV files found in directory: {ldap_dir}")
print(f"Found {len(ldap_files)} LDAP files: {ldap_files}")

Found 18 LDAP files: ['/content/LDAP/2009-12.csv', '/content/LDAP/2010-01.csv', '/content/LDAP/2010-02.csv', '/content/LDAP/2010-03.csv', '/content/LDAP/2010-04.csv', '/content/LDAP/2010-05.csv', '/content/LDAP/2010-06.csv', '/content/LDAP/2010-07.csv', '/content/LDAP/2010-08.csv', '/content/LDAP/2010-09.csv', '/content/LDAP/2010-10.csv', '/content/LDAP/2010-11.csv', '/content/LDAP/2010-12.csv', '/content/LDAP/2011-01.csv', '/content/LDAP/2011-02.csv', '/content/LDAP/2011-03.csv', '/content/LDAP/2011-04.csv', '/content/LDAP/2011-05.csv']


In [ ]:
# Function to extract YYYY-MM from filename
def extract_month_from_filename(filename):
    match = re.search(r'(\d{4}-\d{2})', filename)
    if match:
        return match.group(1)
    else:
        raise ValueError(f"Could not extract YYYY-MM from filename: {filename}")

# Function to clean prefixes, skipping null values
def clean_prefix(value):
    if pd.isna(value):  # Skip if value is NaN
        return value
    return re.sub(r'^\d+\s*-\s*', '', str(value))

In [ ]:
ldap_dfs = []
for file in ldap_files:
    df = pd.read_csv(file)

    # Verify expected columns (excluding 'month' which we add)
    expected_columns = ['employee_name', 'user_id', 'email', 'role', 'business_unit',
                       'functional_unit', 'department', 'team', 'supervisor']
    missing_cols = [col for col in expected_columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"File {file} is missing columns: {missing_cols}")

    # Clean prefixes, skipping null values
    for col in ['functional_unit', 'department', 'team']:
        df[col] = df[col].apply(clean_prefix)

    # Extract month
    month = extract_month_from_filename(os.path.basename(file))
    df['month'] = month
    ldap_dfs.append(df)

In [ ]:
ldap_combined = pd.concat(ldap_dfs, ignore_index=True)

In [ ]:
ldap_combined.isnull().sum()

,0
employee_name,0
user_id,0
email,0
role,0
business_unit,0
functional_unit,36
department,252
team,2194
supervisor,18
month,0


In [ ]:
ldap_combined.fillna({
    'role': 'Unknown',
    'functional_unit': 'Unknown',
    'department': 'Unknown',
    'team': 'TopLevel',
    'supervisor': 'None'
}, inplace=True)

In [ ]:
ldap_combined.isnull().sum()

,0
employee_name,0
user_id,0
email,0
role,0
business_unit,0
functional_unit,0
department,0
team,0
supervisor,0
month,0


In [ ]:
# Convert 'month' to datetime
ldap_combined['month'] = pd.to_datetime(ldap_combined['month'], format='%Y-%m')

# Add month_year for merging
ldap_combined['month_year'] = ldap_combined['month'].dt.strftime('%Y-%m')

In [ ]:
df_features['date_only'] = pd.to_datetime(df_features['date_only'])
df_features['month_year'] = df_features['date_only'].dt.strftime('%Y-%m')

In [ ]:
df_features['user'] = df_features['user'].astype(str).str.strip().str.upper()

In [ ]:
df_features

,user,date_only,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,emails_sent,bcc_flag,keyword_richness,month_year
0,AAE0190,2010-01-04,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,-0.759447,1.141349,-0.583959,0.386906,2010-01
1,AAE0190,2010-01-05,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.711501,-0.233484,0.952298,-0.583959,0.374986,2010-01
2,AAE0190,2010-01-06,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.064106,1.141349,-0.583959,1.463670,2010-01
3,AAE0190,2010-01-07,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.875382,-0.179997,1.141349,-0.583959,0.323333,2010-01
4,AAE0190,2010-01-08,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.607898,0.952298,-0.583959,0.001496,2010-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330280,ZSL0305,2011-05-10,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,-1.107117,-1.316319,-0.583959,-1.317639,2011-05
330281,ZSL0305,2011-05-11,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.200445,2.137805,-1.316319,-0.583959,-1.285853,2011-05
330282,ZSL0305,2011-05-12,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.091191,3.760267,-1.316319,-0.583959,-1.289826,2011-05
330283,ZSL0305,2011-05-13,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,0.515344,-1.316319,-0.583959,-1.361345,2011-05


In [ ]:
df_features.isnull().sum()

,0
user,0
date_only,0
after_hours_logon_count,0
total_logon_count,0
device_connects,0
avg_content_word_count,0
text_files_accessed,0
files_accessed,0
total_recipients,0
external_ratio,0


In [ ]:
# Convert ldap_combined['month'] to 'YYYY-MM' format
ldap_combined['month_year'] = ldap_combined['month'].dt.strftime('%Y-%m')

In [ ]:
ldap_combined

,employee_name,user_id,email,role,business_unit,functional_unit,department,team,supervisor,month,month_year
0,Calvin Edan Love,CEL0561,Calvin.Edan.Love@dtaa.com,ComputerProgrammer,1,ResearchAndEngineering,SoftwareManagement,Software,Stephanie Briar Harrington,2009-12-01,2009-12
1,Christine Reagan Deleon,CRD0624,Christine.Reagan.Deleon@dtaa.com,Salesman,1,SalesAndMarketing,Sales,RegionalSales,Winter Veda Burks,2009-12-01,2009-12
2,Jade Felicia Caldwell,JFC0557,Jade.Felicia.Caldwell@dtaa.com,SoftwareEngineer,1,ResearchAndEngineering,SoftwareManagement,Software,Stephanie Briar Harrington,2009-12-01,2009-12
3,Aquila Stewart Dejesus,ASD0577,Aquila.Stewart.Dejesus@dtaa.com,ProductionLineWorker,1,Manufacturing,Assembly,AssemblyDept,Whilemina Pandora England,2009-12-01,2009-12
4,Micah Abdul Rojas,MAR0955,Micah.Abdul.Rojas@dtaa.com,ProductionLineWorker,1,Manufacturing,Assembly,AssemblyDept,Sandra Beverly Diaz,2009-12-01,2009-12
...,...,...,...,...,...,...,...,...,...,...,...
16738,Isadora Blaine Shepherd,IBS0836,Isadora.Blaine.Shepherd@dtaa.com,ProductionLineWorker,1,Manufacturing,Assembly,AssemblyDept,Amy Lucy Chen,2011-05-01,2011-05
16739,Dylan Tiger Terry,DTT0076,Dylan.Tiger.Terry@dtaa.com,ChiefEngineer,1,ResearchAndEngineering,Engineering,ElectricalEngineering,Declan Lester Moss,2011-05-01,2011-05
16740,Price Rashad Mullen,PRM0740,Price.Rashad.Mullen@dtaa.com,Salesman,1,SalesAndMarketing,Sales,RegionalSales,Lysandra Chastity Brennan,2011-05-01,2011-05
16741,Florence Gloria Whitney,FGW0853,Florence.Gloria.Whitney@dtaa.com,Salesman,1,SalesAndMarketing,Sales,RegionalSales,Hashim Damon Dudley,2011-05-01,2011-05


In [ ]:
ldap_combined.isnull().sum()

,0
employee_name,0
user_id,0
email,0
role,0
business_unit,0
functional_unit,0
department,0
team,0
supervisor,0
month,0


In [ ]:
ldap_combined[ldap_combined['role'] == 'President']

,employee_name,user_id,email,role,business_unit,functional_unit,department,team,supervisor,month,month_year
809,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2009-12-01,2009-12
1809,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2010-01-01,2010-01
2801,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2010-02-01,2010-02
3784,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2010-03-01,2010-03
4764,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2010-04-01,2010-04
5738,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2010-05-01,2010-05
6706,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2010-06-01,2010-06
7668,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2010-07-01,2010-07
8619,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2010-08-01,2010-08
9555,Mona Susan Shannon,MSS0001,Mona.Susan.Shannon@dtaa.com,President,1,Unknown,Unknown,TopLevel,None,2010-09-01,2010-09


In [ ]:
# Clean user IDs in both dataframes
df_features['user'] = df_features['user'].astype(str).str.strip().str.upper()
ldap_combined['user_id'] = ldap_combined['user_id'].astype(str).str.strip().str.upper()

In [ ]:
last_seen = ldap_combined.groupby('user_id')['month'].max().reset_index()
last_seen.rename(columns={'month': 'last_seen_month'}, inplace=True)

In [ ]:
np.sort(last_seen['last_seen_month'].unique())

array(['2010-01-01T00:00:00.000000000', '2010-02-01T00:00:00.000000000',
       '2010-03-01T00:00:00.000000000', '2010-04-01T00:00:00.000000000',
       '2010-05-01T00:00:00.000000000', '2010-06-01T00:00:00.000000000',
       '2010-07-01T00:00:00.000000000', '2010-08-01T00:00:00.000000000',
       '2010-09-01T00:00:00.000000000', '2010-10-01T00:00:00.000000000',
       '2010-11-01T00:00:00.000000000', '2010-12-01T00:00:00.000000000',
       '2011-01-01T00:00:00.000000000', '2011-02-01T00:00:00.000000000',
       '2011-03-01T00:00:00.000000000', '2011-04-01T00:00:00.000000000',
       '2011-05-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [ ]:
# Merge this information with df_features
df_features = df_features.merge(
    last_seen[['user_id', 'last_seen_month']],
    left_on='user',
    right_on='user_id',
    how='left'
)
df_features.drop(columns=['user_id'], inplace=True)

In [ ]:
df_features

,user,date_only,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,emails_sent,bcc_flag,keyword_richness,month_year,last_seen_month,has_left
0,AAE0190,2010-01-04,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,-0.759447,1.141349,-0.583959,0.386906,2010-01,2011-05-01,0
1,AAE0190,2010-01-05,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.711501,-0.233484,0.952298,-0.583959,0.374986,2010-01,2011-05-01,0
2,AAE0190,2010-01-06,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.064106,1.141349,-0.583959,1.463670,2010-01,2011-05-01,0
3,AAE0190,2010-01-07,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.875382,-0.179997,1.141349,-0.583959,0.323333,2010-01,2011-05-01,0
4,AAE0190,2010-01-08,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.607898,0.952298,-0.583959,0.001496,2010-01,2011-05-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330280,ZSL0305,2011-05-10,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,-1.107117,-1.316319,-0.583959,-1.317639,2011-05,2011-05-01,1
330281,ZSL0305,2011-05-11,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.200445,2.137805,-1.316319,-0.583959,-1.285853,2011-05,2011-05-01,1
330282,ZSL0305,2011-05-12,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.091191,3.760267,-1.316319,-0.583959,-1.289826,2011-05,2011-05-01,1
330283,ZSL0305,2011-05-13,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,0.515344,-1.316319,-0.583959,-1.361345,2011-05,2011-05-01,1


In [ ]:
df_features.drop(columns = {'last_seen_month_y'}, inplace=True)
df_features.rename(columns = {'last_seen_month_x': 'last_seen_month'}, inplace=True)

KeyError: "['last_seen_month_y'] not found in axis"

In [ ]:
df_features['last_seen_month'] = pd.to_datetime(df_features['last_seen_month'])

In [ ]:
df_features['has_left'] = (df_features['date_only'] > df_features['last_seen_month']).astype(int)

In [ ]:
df_features

,user,date_only,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,emails_sent,bcc_flag,keyword_richness,month_year,last_seen_month,has_left
0,AAE0190,2010-01-04,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,-0.759447,1.141349,-0.583959,0.386906,2010-01,2011-05-01,0
1,AAE0190,2010-01-05,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.711501,-0.233484,0.952298,-0.583959,0.374986,2010-01,2011-05-01,0
2,AAE0190,2010-01-06,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.064106,1.141349,-0.583959,1.463670,2010-01,2011-05-01,0
3,AAE0190,2010-01-07,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.875382,-0.179997,1.141349,-0.583959,0.323333,2010-01,2011-05-01,0
4,AAE0190,2010-01-08,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.607898,0.952298,-0.583959,0.001496,2010-01,2011-05-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330280,ZSL0305,2011-05-10,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,-1.107117,-1.316319,-0.583959,-1.317639,2011-05,2011-05-01,1
330281,ZSL0305,2011-05-11,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.200445,2.137805,-1.316319,-0.583959,-1.285853,2011-05,2011-05-01,1
330282,ZSL0305,2011-05-12,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.091191,3.760267,-1.316319,-0.583959,-1.289826,2011-05,2011-05-01,1
330283,ZSL0305,2011-05-13,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,0.515344,-1.316319,-0.583959,-1.361345,2011-05,2011-05-01,1


In [ ]:
df_features.drop(columns=['last_seen_month'], inplace=True)

In [ ]:
df_features_active = df_features[df_features['has_left'] == 0].merge(
    ldap_combined[['user_id', 'month_year', 'role', 'functional_unit', 'department', 'team', 'supervisor']],
    left_on=['user', 'month_year'],
    right_on=['user_id', 'month_year'],
    how='left'
)

In [ ]:
df_features_active.columns

Index(['user', 'date_only', 'after_hours_logon_count', 'total_logon_count',
       'device_connects', 'avg_content_word_count', 'text_files_accessed',
       'files_accessed', 'total_recipients', 'external_ratio', 'emails_sent',
       'bcc_flag', 'keyword_richness', 'month_year', 'last_seen_month',
       'has_left', 'user_id', 'role', 'functional_unit', 'department', 'team',
       'supervisor'],
      dtype='object')

In [ ]:
df_features_active.isnull().sum()

,0
user,0
date_only,0
after_hours_logon_count,0
total_logon_count,0
device_connects,0
avg_content_word_count,0
text_files_accessed,0
files_accessed,0
total_recipients,0
external_ratio,0


In [ ]:
df_features_active.shape

(315781, 22)

In [ ]:
df_features_post = df_features[df_features['has_left'] == 1].copy()
if not df_features_post.empty:
    # Find the last LDAP data for each user
    last_ldap_data = ldap_combined.sort_values('month').groupby('user_id').last().reset_index()
    df_features_post = df_features_post.merge(
        last_ldap_data[['user_id', 'role', 'functional_unit', 'department', 'team', 'supervisor']],
        left_on='user',
        right_on='user_id',
        how='left'
    )

In [ ]:
df_features_post

,user,date_only,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,...,keyword_richness,month_year,last_seen_month,has_left,user_id,role,functional_unit,department,team,supervisor
0,AAE0190,2011-05-02,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,0.515344,...,0.704770,2011-05,2011-05-01,1,AAE0190,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
1,AAE0190,2011-05-03,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,-0.179997,...,1.300765,2011-05,2011-05-01,1,AAE0190,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
2,AAE0190,2011-05-04,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.602247,0.399454,...,1.185539,2011-05,2011-05-01,1,AAE0190,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
3,AAE0190,2011-05-05,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.547620,-0.295887,...,-0.113730,2011-05,2011-05-01,1,AAE0190,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
4,AAE0190,2011-05-06,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,1.257772,0.283564,...,0.994821,2011-05,2011-05-01,1,AAE0190,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14499,ZSL0305,2011-05-10,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,-1.107117,...,-1.317639,2011-05,2011-05-01,1,ZSL0305,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price
14500,ZSL0305,2011-05-11,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.200445,2.137805,...,-1.285853,2011-05,2011-05-01,1,ZSL0305,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price
14501,ZSL0305,2011-05-12,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.091191,3.760267,...,-1.289826,2011-05,2011-05-01,1,ZSL0305,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price
14502,ZSL0305,2011-05-13,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,0.515344,...,-1.361345,2011-05,2011-05-01,1,ZSL0305,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price


In [ ]:
df_features_post.isnull().sum()

,0
user,0
date_only,0
after_hours_logon_count,0
total_logon_count,0
device_connects,0
avg_content_word_count,0
text_files_accessed,0
files_accessed,0
total_recipients,0
external_ratio,0


In [ ]:
df_features.isnull().sum()

,0
user,0
date_only,0
after_hours_logon_count,0
total_logon_count,0
device_connects,0
avg_content_word_count,0
text_files_accessed,0
files_accessed,0
total_recipients,0
external_ratio,0


In [ ]:
df_features = pd.concat([df_features_active, df_features_post], ignore_index=True)

# Step 4: Drop redundant user_id column
df_features.drop(columns=['user_id'], inplace=True)

In [ ]:
df_features.columns

Index(['user', 'date_only', 'after_hours_logon_count', 'total_logon_count',
       'device_connects', 'avg_content_word_count', 'text_files_accessed',
       'files_accessed', 'total_recipients', 'external_ratio', 'emails_sent',
       'bcc_flag', 'keyword_richness', 'month_year', 'last_seen_month',
       'has_left', 'role', 'functional_unit', 'department', 'team',
       'supervisor'],
      dtype='object')

In [ ]:
print("NaNs in final df_features after concat:")
print(df_features[['role', 'functional_unit', 'department', 'team', 'supervisor']].isna().sum())

NaNs in final df_features after concat:
role               0
functional_unit    0
department         0
team               0
supervisor         0
dtype: int64


In [ ]:
df_features

,user,date_only,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,...,bcc_flag,keyword_richness,month_year,last_seen_month,has_left,role,functional_unit,department,team,supervisor
0,AAE0190,2010-01-04,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,-0.759447,...,-0.583959,0.386906,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
1,AAE0190,2010-01-05,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.711501,-0.233484,...,-0.583959,0.374986,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
2,AAE0190,2010-01-06,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.064106,...,-0.583959,1.463670,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
3,AAE0190,2010-01-07,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.875382,-0.179997,...,-0.583959,0.323333,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
4,AAE0190,2010-01-08,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.607898,...,-0.583959,0.001496,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330280,ZSL0305,2011-05-10,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,-1.107117,...,-0.583959,-1.317639,2011-05,2011-05-01,1,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price
330281,ZSL0305,2011-05-11,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.200445,2.137805,...,-0.583959,-1.285853,2011-05,2011-05-01,1,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price
330282,ZSL0305,2011-05-12,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.091191,3.760267,...,-0.583959,-1.289826,2011-05,2011-05-01,1,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price
330283,ZSL0305,2011-05-13,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,0.515344,...,-0.583959,-1.361345,2011-05,2011-05-01,1,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price


In [ ]:
# Verify no rows have role as NaN
assert df_features['role'].isna().sum() == 0, "Some rows still have role as NaN"

In [ ]:
# Create role description, excluding placeholder values
def create_role_description(row, include_has_left=True):
    placeholders = {'Unknown', 'TopLevel', 'None'}
    parts = []
    if row['role'] not in placeholders:
        parts.append(row['role'])
    org_context = []
    if row['functional_unit'] not in placeholders:
        org_context.append(row['functional_unit'])
    if row['department'] not in placeholders:
        org_context.append(f"{row['department']} team")
    if org_context:
        parts.append(" in " + ", ".join(org_context))
    if row['supervisor'] not in placeholders:
        parts.append(f"supervised by {row['supervisor']}")
    if include_has_left and row['has_left'] == 1:
        parts.append("(has left the company)")
    return " ".join(parts) if parts else "Unknown Role"

# Apply to df_features (optionally include has_left in the description)
df_features['role_text'] = df_features.apply(lambda row: create_role_description(row, include_has_left=True), axis=1)

In [ ]:
df_features.to_csv('df_with_roletext.csv')

In [ ]:
#download df_features
from google.colab import files
files.download('df_with_roletext.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_features.isnull().sum()

,0
user,0
date_only,0
after_hours_logon_count,0
total_logon_count,0
device_connects,0
avg_content_word_count,0
text_files_accessed,0
files_accessed,0
total_recipients,0
external_ratio,0


In [ ]:
df_features = pd.read_csv('/content/df_with_roletext.csv')

In [ ]:
df_features = df_features.drop(columns = {'Unnamed: 0'})

In [ ]:
df_features

,user,date_only,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,...,keyword_richness,month_year,last_seen_month,has_left,role,functional_unit,department,team,supervisor,role_text
0,AAE0190,2010-01-04,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,-0.759447,...,0.386906,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard,"Manager in PurchasingAndContracts, Purchasing..."
1,AAE0190,2010-01-05,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.711501,-0.233484,...,0.374986,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard,"Manager in PurchasingAndContracts, Purchasing..."
2,AAE0190,2010-01-06,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.064106,...,1.463670,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard,"Manager in PurchasingAndContracts, Purchasing..."
3,AAE0190,2010-01-07,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.875382,-0.179997,...,0.323333,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard,"Manager in PurchasingAndContracts, Purchasing..."
4,AAE0190,2010-01-08,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.607898,...,0.001496,2010-01,2011-05-01,0,Manager,PurchasingAndContracts,Purchasing,TopLevel,Kirby Bo Pollard,"Manager in PurchasingAndContracts, Purchasing..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330280,ZSL0305,2011-05-10,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,-1.107117,...,-1.317639,2011-05,2011-05-01,1,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price,"ProductionLineWorker in Manufacturing, Assemb..."
330281,ZSL0305,2011-05-11,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.200445,2.137805,...,-1.285853,2011-05,2011-05-01,1,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price,"ProductionLineWorker in Manufacturing, Assemb..."
330282,ZSL0305,2011-05-12,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.091191,3.760267,...,-1.289826,2011-05,2011-05-01,1,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price,"ProductionLineWorker in Manufacturing, Assemb..."
330283,ZSL0305,2011-05-13,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,0.515344,...,-1.361345,2011-05,2011-05-01,1,ProductionLineWorker,Manufacturing,Assembly,AssemblyDept,Shannon Chantale Price,"ProductionLineWorker in Manufacturing, Assemb..."


In [ ]:
df_features['role_text'].unique()

array(['Manager  in PurchasingAndContracts, Purchasing team supervised by Kirby Bo Pollard',
       'Technician  in SalesAndMarketing, FieldService team supervised by Burke Randall Burnett',
       'Physicist  in ResearchAndEngineering, Research team supervised by Sasha Britanney Quinn',
       'ProductionLineWorker  in Manufacturing, Assembly team supervised by Alan Benjamin Holder',
       'Technician  in SalesAndMarketing, FieldService team supervised by Hedwig Regina Livingston',
       'ProductionLineWorker  in Manufacturing, Assembly team supervised by Amy Lucy Chen',
       'Technician  in ResearchAndEngineering, Engineering team supervised by Cameron Noel Elliott',
       'ProductionLineWorker  in Manufacturing, Assembly team supervised by Armando Albert Walter',
       'AssemblySupervisor  in Manufacturing, Assembly team supervised by Delilah Iliana Wilkinson',
       'Salesman  in SalesAndMarketing, Sales team supervised by Meredith Ainsley Wolf',
       'ElectricalEngineer  

In [ ]:
# Step 1: Get unique role_text values
unique_role_texts = df_features['role_text'].unique()
print(f"Number of unique role_text values: {len(unique_role_texts)}")

Number of unique role_text values: 347


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # 384-dimensional embeddings
role_embeddings_dict = {
    role_text: model.encode(role_text) for role_text in unique_role_texts
}

KeyboardInterrupt: 

In [ ]:
#save role_embeddings
import pickle
with open('role_embeddings.pkl', 'wb') as f:
    pickle.dump(role_embeddings_dict, f)

#download
from google.colab import files
files.download('role_embeddings.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
role_embeddings_dict = {}
with open('role_embeddings.pkl', 'rb') as f:
    role_embeddings_dict = pickle.load(f)

In [ ]:
embedding_dim = 384
role_embeddings = df_features['role_text'].map(role_embeddings_dict)
role_embeddings_df = pd.DataFrame(
    list(role_embeddings.values),
    columns=[f'role_emb_{i}' for i in range(embedding_dim)],
    index=df_features.index
)

In [ ]:
role_embeddings_df

,role_emb_0,role_emb_1,role_emb_2,role_emb_3,role_emb_4,role_emb_5,role_emb_6,role_emb_7,role_emb_8,role_emb_9,...,role_emb_374,role_emb_375,role_emb_376,role_emb_377,role_emb_378,role_emb_379,role_emb_380,role_emb_381,role_emb_382,role_emb_383
0,-0.051864,-0.010984,-0.060230,-0.058794,-0.026777,0.020765,0.066316,0.043808,-0.031827,0.010012,...,0.042039,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396
1,-0.051864,-0.010984,-0.060230,-0.058794,-0.026777,0.020765,0.066316,0.043808,-0.031827,0.010012,...,0.042039,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396
2,-0.051864,-0.010984,-0.060230,-0.058794,-0.026777,0.020765,0.066316,0.043808,-0.031827,0.010012,...,0.042039,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396
3,-0.051864,-0.010984,-0.060230,-0.058794,-0.026777,0.020765,0.066316,0.043808,-0.031827,0.010012,...,0.042039,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396
4,-0.051864,-0.010984,-0.060230,-0.058794,-0.026777,0.020765,0.066316,0.043808,-0.031827,0.010012,...,0.042039,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330280,-0.117985,-0.035809,-0.020248,0.015552,-0.062332,-0.016593,-0.004174,-0.006100,-0.012698,-0.016185,...,-0.020004,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095
330281,-0.117985,-0.035809,-0.020248,0.015552,-0.062332,-0.016593,-0.004174,-0.006100,-0.012698,-0.016185,...,-0.020004,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095
330282,-0.117985,-0.035809,-0.020248,0.015552,-0.062332,-0.016593,-0.004174,-0.006100,-0.012698,-0.016185,...,-0.020004,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095
330283,-0.117985,-0.035809,-0.020248,0.015552,-0.062332,-0.016593,-0.004174,-0.006100,-0.012698,-0.016185,...,-0.020004,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095


In [ ]:
role_embeddings_df.to_csv('role_embeddings.csv')

In [ ]:
#download
files.download('role_embeddings.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_features = pd.concat([df_features, role_embeddings_df], axis=1)

# Verify the new columns
print("New columns added for embeddings:")
print(df_features[[f'role_emb_{i}' for i in range(5)]].head())

New columns added for embeddings:
   role_emb_0  role_emb_1  role_emb_2  role_emb_3  role_emb_4
0   -0.051864   -0.010984    -0.06023   -0.058794   -0.026777
1   -0.051864   -0.010984    -0.06023   -0.058794   -0.026777
2   -0.051864   -0.010984    -0.06023   -0.058794   -0.026777
3   -0.051864   -0.010984    -0.06023   -0.058794   -0.026777
4   -0.051864   -0.010984    -0.06023   -0.058794   -0.026777


In [ ]:
df_features.to_csv('df_with_roleemb.csv')

In [ ]:
files.download('df_with_roleemb.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_features.columns[13:]

Index(['month_year', 'last_seen_month', 'has_left', 'role', 'functional_unit',
       'department', 'team', 'supervisor', 'role_text', 'role_emb_0',
       ...
       'role_emb_374', 'role_emb_375', 'role_emb_376', 'role_emb_377',
       'role_emb_378', 'role_emb_379', 'role_emb_380', 'role_emb_381',
       'role_emb_382', 'role_emb_383'],
      dtype='object', length=393)

In [ ]:
feature_cols = ['after_hours_logon_count', 'total_logon_count', 'device_connects',
       'avg_content_word_count', 'text_files_accessed', 'files_accessed',
       'total_recipients', 'external_ratio', 'emails_sent', 'bcc_flag',
       'keyword_richness']

In [ ]:
type(feature_cols)

list

In [ ]:
embedding_cols = [f'role_emb_{i}' for i in range(384)]

In [ ]:
clustering_features = df_features.groupby('user')[feature_cols + embedding_cols].mean().reset_index()

In [ ]:
clustering_features.columns

Index(['user', 'after_hours_logon_count', 'total_logon_count',
       'device_connects', 'avg_content_word_count', 'text_files_accessed',
       'files_accessed', 'total_recipients', 'external_ratio', 'emails_sent',
       ...
       'role_emb_374', 'role_emb_375', 'role_emb_376', 'role_emb_377',
       'role_emb_378', 'role_emb_379', 'role_emb_380', 'role_emb_381',
       'role_emb_382', 'role_emb_383'],
      dtype='object', length=396)

In [ ]:
scaler = StandardScaler()
clustering_features_scaled = scaler.fit_transform(clustering_features.drop(columns=['user']))

In [ ]:
!pip install umap-learn # Install the umap-learn package

import umap.umap_ as umap # Import UMAP from the umap.umap_ module

In [ ]:
# Apply UMAP for dimensionality reduction
reducer = umap.UMAP(n_components=10, random_state=42)
clustering_features_reduced = reducer.fit_transform(clustering_features_scaled)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5)
cluster_labels = clusterer.fit_predict(clustering_features_reduced)
clustering_features['cluster'] = cluster_labels

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
np.sort(clustering_features['cluster'].unique())

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])

In [ ]:
# Merge cluster labels back to df_features
cluster_mapping = clustering_features[['user', 'cluster']]
df_features = df_features.merge(cluster_mapping, on='user', how='left')

In [ ]:
# Check cluster distribution
print("Cluster distribution:")
print(df_features['cluster'].value_counts())

Cluster distribution:
cluster
 10    25649
 22    23293
 23    19997
 7     15404
 26    14842
 15    13421
 19    12893
 13    11327
 28    11316
 5     10382
 21    10293
 8     10198
 3     10155
 25    10134
 11    10019
 0      9989
 20     9860
 17     9711
 12     9677
 4      9626
 1      9596
 2      8999
 24     8796
 27     7684
 9      7497
 14     7342
 16     6494
 18     6412
 6      6364
-1      2915
Name: count, dtype: int64


In [ ]:
len(df_features['cluster'].unique())

30

In [ ]:
noise_mask = clustering_features['cluster'] == -1
noise_points = clustering_features_reduced[noise_mask]

In [ ]:
noise_points

array([[12.819303 ,  5.923857 ,  8.450333 ,  1.8044581, 12.171851 ,
         4.0750647,  8.668757 ,  8.636234 ,  6.5092945,  5.3434825],
       [12.844921 ,  5.8983035,  8.424815 ,  1.7788936, 12.197652 ,
         4.1005177,  8.643255 ,  8.610871 ,  6.5349126,  5.342436 ],
       [12.834232 ,  5.9090877,  8.435238 ,  1.7894847, 12.185971 ,
         4.0905   ,  8.653414 ,  8.62081  ,  6.523803 ,  5.3435454],
       [12.805078 ,  5.9381557,  8.464635 ,  1.8187308, 12.157717 ,
         4.0607586,  8.683057 ,  8.650575 ,  6.495044 ,  5.3449907],
       [12.832883 ,  5.9103904,  8.436865 ,  1.7909269, 12.185485 ,
         4.0886073,  8.655251 ,  8.622752 ,  6.5228   ,  5.3435855],
       [11.086382 ,  1.9830856,  1.3202417,  1.0509632,  7.2765355,
         7.7020297,  4.2792764,  3.5758278,  6.345648 ,  4.6491513],
       [12.832724 ,  5.910563 ,  8.437052 ,  1.7910937, 12.18537  ,
         4.0884356,  8.655437 ,  8.622944 ,  6.5226555,  5.343591 ],
       [12.874024 ,  5.8692856,  8.395807

In [ ]:
non_noise_points = clustering_features_reduced[~noise_mask]
non_noise_labels = clustering_features['cluster'][~noise_mask]

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

In [ ]:
if len(noise_points) > 0:
    nearest_clusters = pairwise_distances_argmin(noise_points, non_noise_points)
    clustering_features.loc[noise_mask, 'cluster'] = non_noise_labels.iloc[nearest_clusters].values

In [ ]:
clustering_features['cluster'].unique()

array([22,  1, 23,  8, 17,  3, 24,  0, 13, 26, 27, 10, 16,  9, 25, 28,  7,
        4, 15,  6,  5,  2, 19, 11, 14, 12, 18, 21, 20])

In [ ]:
df_features.drop(columns=['cluster'], inplace=True, errors='ignore')
cluster_mapping = clustering_features[['user', 'cluster']]
df_features = df_features.merge(cluster_mapping, on='user', how='left')

In [ ]:
clustering_features

,user,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,emails_sent,...,role_emb_375,role_emb_376,role_emb_377,role_emb_378,role_emb_379,role_emb_380,role_emb_381,role_emb_382,role_emb_383,cluster
0,AAE0190,-0.737332,-0.534060,-0.331274,-0.396378,-0.285834,-0.287121,0.794705,0.008093,1.068679,...,-0.033895,0.008010,-0.124472,-0.022922,0.003871,-0.073125,-0.058027,0.041217,0.031497,22
1,AAF0535,-0.737332,-0.534060,0.803965,1.495979,0.176853,0.176168,-0.805398,-0.241640,-0.952049,...,-0.016869,0.004899,-0.036197,-0.001904,0.011525,-0.076935,-0.021924,0.024770,-0.000101,1
2,AAF0791,-0.737332,-0.534060,-0.331274,-0.396378,-0.285834,-0.287121,0.009718,0.416540,0.140360,...,0.128797,-0.028075,-0.087717,-0.086040,-0.013990,0.063994,-0.016303,0.018433,-0.011462,23
3,AAL0706,0.607318,-0.534060,-0.331274,-0.396378,-0.285834,-0.287121,-1.257756,0.112073,-1.321782,...,0.027279,-0.009952,-0.070973,0.029382,0.045405,0.058122,-0.013502,0.052824,-0.030660,8
4,AAM0658,-0.707450,-0.511710,-0.314533,-0.355355,-0.255175,-0.257799,-0.943334,0.008175,-0.951659,...,-0.013633,0.015627,-0.027123,-0.036667,0.033276,-0.055613,-0.021308,-0.014965,0.002161,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ZKS0899,-0.737332,-0.392353,4.320250,2.441526,4.974817,4.964060,-0.798320,0.228522,-0.950783,...,0.004741,0.004870,0.018458,0.010590,0.049654,-0.059849,0.014665,-0.004644,0.005600,2
996,ZMC0284,-0.737332,-0.534060,-0.331274,-0.396378,-0.285834,-0.287121,-0.005439,-0.347453,0.132165,...,0.094696,0.013450,-0.007525,-0.053137,-0.024908,0.022692,0.009161,0.099065,0.011801,14
997,ZSB0649,-0.737332,-0.534060,-0.331274,-0.396378,-0.285834,-0.287121,-0.935677,-0.136454,-0.949690,...,-0.012567,0.018138,-0.026643,-0.034717,0.033361,-0.055802,-0.019173,-0.016482,0.001446,17
998,ZSK0258,0.607318,-0.534060,-0.331274,-0.396378,-0.285834,-0.287121,-0.016175,0.053216,0.133804,...,0.082026,-0.108026,-0.001045,-0.087308,-0.019063,0.043028,0.045335,0.035010,-0.026942,27


In [ ]:
df_features

,user,date_only,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,...,role_emb_375,role_emb_376,role_emb_377,role_emb_378,role_emb_379,role_emb_380,role_emb_381,role_emb_382,role_emb_383,cluster
0,AAE0190,2010-01-04,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,-0.759447,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
1,AAE0190,2010-01-05,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.711501,-0.233484,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
2,AAE0190,2010-01-06,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.064106,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
3,AAE0190,2010-01-07,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.875382,-0.179997,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
4,AAE0190,2010-01-08,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.607898,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330280,ZSL0305,2011-05-10,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,-1.107117,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11
330281,ZSL0305,2011-05-11,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.200445,2.137805,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11
330282,ZSL0305,2011-05-12,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.091191,3.760267,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11
330283,ZSL0305,2011-05-13,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,0.515344,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11


In [ ]:
df_features.to_csv('df_with_cluster.csv')

In [ ]:
#download
files.download('df_with_cluster.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_features

,user,date_only,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,...,role_emb_375,role_emb_376,role_emb_377,role_emb_378,role_emb_379,role_emb_380,role_emb_381,role_emb_382,role_emb_383,cluster
0,AAE0190,2010-01-04,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,-0.759447,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
1,AAE0190,2010-01-05,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.711501,-0.233484,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
2,AAE0190,2010-01-06,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.064106,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
3,AAE0190,2010-01-07,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.875382,-0.179997,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
4,AAE0190,2010-01-08,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.607898,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330280,ZSL0305,2011-05-10,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,-1.107117,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11
330281,ZSL0305,2011-05-11,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.200445,2.137805,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11
330282,ZSL0305,2011-05-12,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.091191,3.760267,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11
330283,ZSL0305,2011-05-13,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,0.515344,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11


In [ ]:
mu_all = np.load('/content/mu_all (1).npy')

In [ ]:
is_anomaly = np.load('/content/is_anomaly.npy')

In [ ]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 MB 2.4 MB/s eta 0:00:00


In [ ]:
# Apply PCA to reduce to 50 dimensions
pca = PCA(n_components=50, random_state=42)
embeddings_reduced = pca.fit_transform(role_embeddings)

In [ ]:
df_features

,user,date_only,after_hours_logon_count,total_logon_count,device_connects,avg_content_word_count,text_files_accessed,files_accessed,total_recipients,external_ratio,...,role_emb_375,role_emb_376,role_emb_377,role_emb_378,role_emb_379,role_emb_380,role_emb_381,role_emb_382,role_emb_383,cluster
0,AAE0190,2010-01-04,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.930009,-0.759447,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
1,AAE0190,2010-01-05,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.711501,-0.233484,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
2,AAE0190,2010-01-06,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.064106,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
3,AAE0190,2010-01-07,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.875382,-0.179997,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
4,AAE0190,2010-01-08,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,0.766128,-0.607898,...,-0.033680,0.008292,-0.124105,-0.022482,0.003725,-0.072560,-0.057533,0.040709,0.031396,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330280,ZSL0305,2011-05-10,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,-1.107117,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11
330281,ZSL0305,2011-05-11,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.200445,2.137805,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11
330282,ZSL0305,2011-05-12,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.091191,3.760267,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11
330283,ZSL0305,2011-05-13,-0.737332,-0.53406,-0.331274,-0.396378,-0.285834,-0.287121,-1.309699,0.515344,...,-0.002039,-0.021126,-0.065286,-0.027105,0.047209,0.064067,-0.099232,0.056521,0.005095,11


In [ ]:
print("Explained variance ratio of the first 50 components:")
print(pca.explained_variance_ratio_.sum())

Explained variance ratio of the first 50 components:
0.96033955


In [ ]:
embedding_dim_reduced = 50
embeddings_reduced_df = pd.DataFrame(
    embeddings_reduced,
    columns=[f'role_emb_pca_{i}' for i in range(embedding_dim_reduced)],
    index=df_features.index
)

In [ ]:
# Remove original embeddings and add reduced embeddings
df_noemb = df_features.drop(columns=embedding_cols)
df_features_new = pd.concat([df_noemb, embeddings_reduced_df], axis=1)

In [ ]:
df_features_new.columns

Index(['user', 'date_only', 'after_hours_logon_count', 'total_logon_count',
       'device_connects', 'avg_content_word_count', 'text_files_accessed',
       'files_accessed', 'total_recipients', 'external_ratio', 'emails_sent',
       'bcc_flag', 'keyword_richness', 'month_year', 'last_seen_month',
       'has_left', 'role', 'functional_unit', 'department', 'team',
       'supervisor', 'role_text', 'cluster', 'role_emb_pca_0',
       'role_emb_pca_1', 'role_emb_pca_2', 'role_emb_pca_3', 'role_emb_pca_4',
       'role_emb_pca_5', 'role_emb_pca_6', 'role_emb_pca_7', 'role_emb_pca_8',
       'role_emb_pca_9', 'role_emb_pca_10', 'role_emb_pca_11',
       'role_emb_pca_12', 'role_emb_pca_13', 'role_emb_pca_14',
       'role_emb_pca_15', 'role_emb_pca_16', 'role_emb_pca_17',
       'role_emb_pca_18', 'role_emb_pca_19', 'role_emb_pca_20',
       'role_emb_pca_21', 'role_emb_pca_22', 'role_emb_pca_23',
       'role_emb_pca_24', 'role_emb_pca_25', 'role_emb_pca_26',
       'role_emb_pca_27', '

In [ ]:
df_features_new.to_csv('df_with_pca.csv')
files.download('df_with_pca.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>